<div class='heading'>
    <div style='float:left;'><h1>CPSC 4300/6300: Applied Data Science</h1></div>
    <img style="float: right; padding-right: 10px; width: 65px" src="https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/images/clemson_paw.png"> </div><br>


## Week 2: Introduction to Pandas

**Clemson University**<br>
**Spring 2024**<br>
**Instructor(s):** Nina Hubig <br>
**TA(s):** Samaneh Zamanifard, Boqiang Li


---

In [12]:
""" RUN THIS CELL TO GET THE RIGHT FORMATTING """
import requests
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/css/cpsc6300.css'
styles = requests.get(css_file).text
HTML(styles)

# Table of Contents 
<ol start="0">
<li> Learning Goals </li>
<li> Loading and Cleaning with Pandas</li>
<li> Parsing and Completing the Dataframe  </li>
<li> Grouping </li>
</ol>

## Learning Goals

About 6,000 odd "best books" were fetched and parsed from [Goodreads](https://www.goodreads.com). The "bestness" of these books came from a proprietary formula used by Goodreads and published as a list on their web site.

We parsed the page for each book and saved data from all these pages in a tabular format as a CSV file. In this lab we'll clean and further parse the data.  We'll then do some exploratory data analysis to answer questions about these best books and popular genres.  


By the end of this lab, you should be able to:

- Load and systematically address missing values, encoded as `NaN` values in our data set, for example, by removing observations associated with these values.
- Parse columns in the dataframe to create new dataframe columns.
- Use groupby to aggregate data on a particular feature column, such as author.

### Basic EDA workflow

The basic workflow is as follows:

1. **Build** a DataFrame from the data (ideally, put all data in this object)
2. **Clean** the DataFrame. It should have the following properties:
    - Each row describes a single object
    - Each column describes a property of that object
    - Columns are numeric whenever appropriate
    - Columns contain atomic properties that cannot be further decomposed
3. Explore **global properties**. Use histograms, scatter plots, and aggregation functions to summarize the data.
4. Explore **group properties**. Use groupby and small multiples to compare subsets of the data.

This process transforms your data into a format which is easier to work with, gives you a basic overview of the data's properties, and likely generates several questions for you to followup in subsequent analysis.

## Part 1: Loading and Cleaning with Pandas 
Read in the `goodreads.csv` file, examine the data, and do any necessary data cleaning. 

Here is a description of the columns (in order) present in this csv file:

```
rating: the average rating on a 1-5 scale achieved by the book
review_count: the number of Goodreads users who reviewed this book
isbn: the ISBN code for the book
booktype: an internal Goodreads identifier for the book
author_url: the Goodreads (relative) URL for the author of the book
year: the year the book was published
genre_urls: a string with '|' separated relative URLS of Goodreads genre pages
dir: a directory identifier internal to the scraping code
rating_count: the number of ratings for this book (this is different from the number of reviews)
name: the name of the book
```

Let us see what issues we find with the data and resolve them.  



----




After loading appropriate libraries


In [8]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

### Cleaning: Reading in the data
We read in and clean the data from `goodreads.csv`.

In [9]:
#Read the data into a dataframe
df = pd.read_csv("goodreads.csv", encoding='utf-8')

#Examine the first few rows of the dataframe
# Examine the first few rows of the dataframe
print(df.head())


   4.40   136455  0439023483  good_reads:book https://www.goodreads.com/author/show/153394.Suzanne_Collins    2008 /genres/young-adult|/genres/science-fiction|/genres/dystopia|/genres/fantasy|/genres/science-fiction|/genres/romance|/genres/adventure|/genres/book-club|/genres/young-adult|/genres/teen|/genres/apocalyptic|/genres/post-apocalyptic|/genres/action                dir01/2767052-the-hunger-games.html    2958974            The Hunger Games (The Hunger Games, #1)
0  4.41  16648.0  0439358078  good_reads:book  https://www.goodreads.com/author/show/1077326....            2003.0  /genres/fantasy|/genres/young-adult|/genres/fi...                                                                                                                                                                                                     dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...  1284478.0  Harry Potter and the Order of the Phoenix (Har...
1  3.56  85746.0  0316015849  good_reads:book  https

Oh dear. That does not quite seem to be right. We are missing the column names. We need to add these in! But what are they?

Here is a list of them in order:

`["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name']`

<div class="exercise"><b>Exercise 1</b></div>
Use these to load the dataframe properly! And then "head" the dataframe... (you will need to look at the read_csv docs)


In [11]:
import pandas as pd

# Load the data from the CSV file
columns = ["rating", "review_count", "isbn", "booktype", "author_url", "year", "genre_urls", "dir", "rating_count", "name"]
df = pd.read_csv("goodreads.csv", encoding='utf-8', names=columns)

# Check the first few rows of the dataframe
print(df.head())


   rating  review_count        isbn         booktype                                         author_url    year                                         genre_urls                                                dir  rating_count                                               name
0    4.40      136455.0  0439023483  good_reads:book  https://www.goodreads.com/author/show/153394.S...  2008.0  /genres/young-adult|/genres/science-fiction|/g...                dir01/2767052-the-hunger-games.html     2958974.0            The Hunger Games (The Hunger Games, #1)
1    4.41       16648.0  0439358078  good_reads:book  https://www.goodreads.com/author/show/1077326....  2003.0  /genres/fantasy|/genres/young-adult|/genres/fi...  dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...     1284478.0  Harry Potter and the Order of the Phoenix (Har...
2    3.56       85746.0  0316015849  good_reads:book  https://www.goodreads.com/author/show/941441.S...  2005.0  /genres/young-adult|/genres/fantasy|/genres/ro... 

### Cleaning: Examing the dataframe - quick checks

We should examine the dataframe to get a overall sense of the content. 

<div class="exercise"><b>Exercise 2</b></div>
Lets check the types of the columns. What do you find?

In [14]:
# Check the data types of the columns
print(df.dtypes)


rating          float64
review_count    float64
isbn             object
booktype         object
author_url       object
year            float64
genre_urls       object
dir              object
rating_count    float64
name             object
dtype: object


*your answer here*

Notice that `review_count` and `rating_counts` are objects instead of ints, and the `year` is a float!

There are a couple more quick sanity checks to perform on the dataframe. 

In [13]:
print(df.shape)
df.columns

(6000, 10)


Index(['rating', 'review_count', 'isbn', 'booktype', 'author_url', 'year', 'genre_urls', 'dir', 'rating_count', 'name'], dtype='object')

### Cleaning: Examining the dataframe - a deeper look

Beyond performing checking some quick general properties of the data frame and looking at the first $n$ rows, we can dig a bit deeper into the values being stored. If you haven't already, check to see if there are any missing values in the data frame.

Let's see for a column which seemed OK to us.

In [17]:
#Get a sense of how many missing values there are in the dataframe.
print(np.sum([df.rating.isnull()]))
print(np.sum([df.review_count.isnull()]))
print(np.sum([df.isbn.isnull()]))
print(np.sum([df.booktype.isnull()]))
print(np.sum([df.author_url.isnull()]))
print(np.sum([df.year.isnull()]))
print(np.sum([df.genre_urls.isnull()]))
print(np.sum([df.dir.isnull()]))
print(np.sum([df.rating_count.isnull()]))
print(np.sum([df.name.isnull()]))

2
2
477
2
2
7
62
0
2
2


In [15]:
#Try to locate where the missing values occur
df[df.rating.isnull()]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
3643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dir37/9658936-harry-potter.html,NaN,NaN
5282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dir53/113138.The_Winner.html,NaN,NaN


How does `pandas` or `numpy` handle missing values when we try to compute with data sets that include them?

We'll now check if any of the other suspicious columns have missing values.  Let's look at `year` and `review_count` first.

One thing you can do is to try and convert to the type you expect the column to be. If something goes wrong, it likely means your data are bad.

Lets test for missing data:

In [16]:
df[df.year.isnull()]

df.year.isnull()
df.shape

(6000, 10)

### Cleaning: Dealing with Missing Values
How should we interpret 'missing' or 'invalid' values in the data (hint: look at where these values occur)? One approach is to simply exclude them from the dataframe. Is this appropriate for all 'missing' or 'invalid' values? 

In [24]:
#Treat the missing or invalid values in your dataframe
####### 

df = df[df.year.notnull()]

Ok so we have done some cleaning. What do things look like now? Notice the float has not yet changed.

In [22]:
df.dtypes

rating          float64
review_count    float64
isbn             object
booktype         object
author_url       object
year            float64
genre_urls       object
dir              object
rating_count    float64
name             object
dtype: object

In [25]:
print(np.sum(df.year.isnull()))
df.shape # We removed seven rows

0


(5993, 10)

<div class="exercise"><b>Exercise 3</b></div>

Ok so lets fix those types. Convert them to ints. If the type conversion fails, we now know we have further problems.

In [31]:
# Convert 'review_count' and 'rating_count' to integers
try:
    df['review_count'] = df['review_count'].astype(int)
    df['rating_count'] = df['rating_count'].astype(int)
    print("Conversion to integers successful for 'review_count' and 'rating_count'")
except ValueError as e:
    print("Conversion failed: ", e)

# Convert 'year' to integer
try:
    df['year'] = df['year'].astype(int)
    print("Conversion to integer successful for 'year'")
except ValueError as e:
    print("Conversion failed: ", e)


Conversion to integers successful for 'review_count' and 'rating_count'
Conversion to integer successful for 'year'


Once you do this, we seem to be good on these columns (no errors in conversion). Lets look:

In [27]:
df.dtypes

rating          float64
review_count      int32
isbn             object
booktype         object
author_url       object
year              int32
genre_urls       object
dir              object
rating_count      int32
name             object
dtype: object

Sweet!

Some of the other colums that should be strings have NaN.

In [32]:
df.loc[df.genre_urls.isnull(), 'genre_urls']=""
df.loc[df.isbn.isnull(), 'isbn']=""

##  Part 2: Parsing and Completing the Data Frame 

We will parse the `author` column from the author_url and `genres` column from the genre_urls. Keep the `genres` column as a string separated by '|'.

We will use panda's `map` to assign new columns to the dataframe.  

Examine an example `author_url` and reason about which sequence of string operations must be performed in order to isolate the author's name.

In [33]:
#Get the first author_url
test_string = df.author_url[0]
test_string

'https://www.goodreads.com/author/show/153394.Suzanne_Collins'

In [34]:
#Test out some string operations to isolate the author name

test_string.split('/')[-1].split('.')[1:][0]

'Suzanne_Collins'

<div class="exercise"><b>Exercise 4</b></div>

Lets wrap the above code into a function which we will then use

In [38]:
# Write a function that accepts an author url and returns the author's name based on your experimentation above
def get_author(url):
    # your code here
    name =  url.split('/')[-1].replace('_', ' ').replace('-', ' ').replace('.', ' ')
    
    return name

In [39]:
#Apply the get_author function to the 'author_url' column using '.map' 
#and add a new column 'author' to store the names
df['author'] = df.author_url.map(get_author)
df.author[0:5]

0    153394 Suzanne Collins
1       1077326 J K Rowling
2    941441 Stephenie Meyer
3           1825 Harper Lee
4          1265 Jane Austen
Name: author, dtype: object


Now parse out the genres from `genre_url`.  

This is a little more complicated because there be more than one genre.


In [ ]:

df.genre_urls.head()

In [40]:
#Examine some examples of genre_urls

#Test out some string operations to isolate the genre name
test_genre_string=df.genre_urls[0]
genres=test_genre_string.strip().split('|')
for e in genres:
    print(e.split('/')[-1])
    "|".join(genres)

young-adult
science-fiction
dystopia
fantasy
science-fiction
romance
adventure
book-club
young-adult
teen
apocalyptic
post-apocalyptic
action


<div class="exercise"><b>Exercise 5</b></div>

Write a function that accepts a genre url and returns the genre name based on your experimentation above



In [42]:
def split_and_join_genres(url):
 # Split the URL by '/' to separate different parts
    parts = url.split('/')

    # Filter out empty strings and irrelevant parts
    genre_parts = [part for part in parts if part and part not in ['genres', 'goodreads']]

    # Join the filtered parts to form the genre name
    genre_name = ', '.join(genre_parts).replace('-', ' ').replace('_', ' ')

    return genre_name

# Example usage
genre_url_example = '/genres/science-fiction/fantasy/dystopia'
genre_name = split_and_join_genres(genre_url_example)
print(genre_name)


science fiction, fantasy, dystopia


Test your function

In [43]:
split_and_join_genres("/genres/young-adult|/genres/science-fiction")

'young adult|, science fiction'

In [44]:
split_and_join_genres("")

''

This question is primarily aimed at graduate students, serving as a critical part of their assignment. However, undergraduate students are encouraged to take this on as an optional bonus challenge.
<div class="exercise"><b>Exercise 6</b></div>
Use map again to create a new "genres" column

In [47]:

# Ensure the split_and_join_genres function is defined as earlier
def split_and_join_genres(url):
    # Your previously defined code here

# Use map to apply the function to each entry in the "genre_urls" column
    df['genres'] = df['genre_urls'].map(split_and_join_genres)

# Display the first few rows of the dataframe to check the new column
    df.head()

Finally, let's pick an author at random so we can see the results of the transformations.  Scroll to see the `author` and `genre` columns that we added to the dataframe.

In [48]:
df[df.author == "Marguerite_Yourcenar"]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name,author


Let us delete the `genre_urls` column.

In [ ]:
del df['genre_urls']

And then save the dataframe out!

In [52]:
df.to_csv("data/cleaned-goodreads.csv", index=False, header=True)

OSError: Cannot save file into a non-existent directory: 'data'

---

## Part 3: Grouping 

It appears that some books were written in negative years!  Print out the observations that correspond to negative years.  What do you notice about these books?  

In [ ]:
df[df.year < 0].name
#These are books written before the Common Era (BCE, equivalent to BC).

We can determine the "best book" by year! For this we use Panda's `groupby()`. `Groupby()` allows grouping a dataframe by any (usually categorical) variable. Would it make sense to ever groupby integer variables? Floating point variables?

In [ ]:
dfgb_author = df.groupby('author')
type(dfgb_author)

Perhaps we want the number of books each author wrote

In [ ]:
dfgb_author.count()

Lots of useless info there. One column should suffice

### Exercise:

- Group the dataframe by `author`. Include the following columns: `rating`, `name`, `author`. For the aggregation of the `name` column which includes the names of the books create a list with the strings containing the name of each book. Make sure that the way you aggregate the rest of the columns make sense! 

- Create a new column with number of books for each author and find the most prolific author!

In [ ]:
###### Before we start : what do we do about these titles where 'name' is unreadable? Try different encodings?
auth_name = 'A_id_al_Qarni'
df[df.author == auth_name].head()

In [ ]:
df[df.author == auth_name].iat[0,8].encode('UTF-16')

In [ ]:
# let's examine the columns we have
df.columns

Create the GroupBy table

In [ ]:
authors = df.copy()
authors = authors[['rating','name','author']].groupby('author').agg({'rating' : np.mean,
                                                                    'name' : '|'.join})

In [ ]:
authors = authors.reset_index()
authors.head()

In [ ]:
# split the column string and make a list of string book names
authors['name'] = authors.name.str.split('|')
authors.head()

In [ ]:
# count the books - create new column
len(authors.name[0])

In [ ]:
authors['num_books'] = authors['name'].str.len()
authors

In [ ]:
# sort for more prolific
authors.sort_values(by='num_books', ascending=False).iloc[0]

#### Winner is Stephen King with 56 books! OMG!!!

Perhaps you want more detailed info...

In [ ]:
dfgb_author[['rating', 'rating_count', 'review_count', 'year']].describe()

You can also access a `groupby` dictionary style.

In [ ]:
ratingdict = {}
for author, subset in dfgb_author:
    ratingdict[author] = (subset['rating'].mean(), subset['rating'].std())
ratingdict

**This question is primarily aimed at graduate students, serving as a critical part of their assignment. However, undergraduate students are encouraged to take this on as an optional bonus challenge.**
<div class="exercise"><b>Exercise 7</b></div>

Lets get the best-rated book(s) for every year in our dataframe.

In [53]:
# Group the dataframe by 'year'
grouped_by_year = df.groupby('year')

# Create a dictionary to store the best-rated book(s) for each year
best_rated_books_per_year = {}

for year, group in grouped_by_year:
    # Find the highest rating in the current year's group
    highest_rating = group['rating'].max()

    # Get all books in the current year's group that have the highest rating
    best_books = group[group['rating'] == highest_rating]

    # Add the result to the dictionary
    best_rated_books_per_year[year] = best_books

# Display the best-rated book(s) for each year
for year, books in best_rated_books_per_year.items():
    print("Year:", year)
    print(books[['name', 'rating']], "\n")



Year: -1500
                       name  rating
1398  The Epic of Gilgamesh     3.6 

Year: -800
                      name  rating
246  The Iliad/The Odyssey    4.01 

Year: -560
                name  rating
1397  Aesop's Fables    4.03 

Year: -512
               name  rating
674  The Art of War    3.92 

Year: -500
                  name  rating
746  The Bhagavad Gita    4.06 

Year: -458
              name  rating
1815  The Oresteia    3.96 

Year: -442
         name  rating
777  Antigone    3.52 

Year: -440
               name  rating
2527  The Histories    3.94 

Year: -431
       name  rating
1428  Medea     3.8 

Year: -429
            name  rating
629  Oedipus Rex    3.64 

Year: -411
                                      name  rating
2078  The History of the Peloponnesian War    3.84 

Year: -400
                name  rating
3133  Complete Works     4.3 

Year: -390
         name  rating
4475  Apology    4.11 

Year: -380
             name  rating
455  The Republic    3.85 


                                                  name  rating
1484  The Harry Potter Collection (Harry Potter, #1-4)    4.63 

Year: 2000
                        name  rating
1385  Standing for Something    4.64 

Year: 2001
                                                  name  rating
1340  A Storm of Swords (A Song of Ice and Fire, #3-1)     4.5 

Year: 2002
            name  rating
3496  Way to Be!    4.62 

Year: 2003
                                  name  rating
4629  The Complete Far Side, 1980-1994    4.66 

Year: 2004
                   name  rating
4651  Ù¾ÛØ±Ù Ú©Ø§Ù
Ù    4.58 

Year: 2005
                               name  rating
249  The Complete Calvin and Hobbes     4.8 

Year: 2006
                                     name  rating
5313  Warriors Boxed Set (Warriors, #1-3)    4.67 

Year: 2007
                 name  rating
1853  ESV Study Bible    4.76 

Year: 2008
                              name  rating
4084  The Absolute Sandman, Vol. 4    4.75 

Year: 2009
 